In [1]:
import numpy as np
import pandas as pd
from collections import Counter

class CandleStick:
    def __init__(self, open, high, low, close):
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self._validate_candle()

    def __repr__(self):
        return f"Candle({self.open}, {self.high}, {self.low}, {self.close})"

    def __str__(self):
        return f"O: {self.open}, H: {self.high}, L: {self.low}, C: {self.close}"

    @property
    def open(self):
        return self._open

    @property
    def high(self):
        return self._high

    @property
    def low(self):
        return self._low

    @property
    def close(self):
        return self._close

    @open.setter
    def open(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("open must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("open must be positive")
        self._open = value

    @high.setter
    def high(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("high must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("high must be positive")
        self._high = value

    @low.setter
    def low(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("low must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("Low must be positive")
        self._low = value

    @close.setter
    def close(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("close must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("close must be positive")
        self._close = value

    def _validate_candle(self):
        if self.open > self.high:
            raise ValueError("open cannot be greater than high")
        if self.open < self.low:
            raise ValueError("open cannot be less than low")
        if self.close > self.high:
            raise ValueError("close cannot be greater than high")
        if self.close < self.low:
            raise ValueError("close cannot be less than low")

    def type(self):
        if self.open < self.close:
            return 'bullish'
        elif self.open > self.close:
            return 'bearish'
        else:
            return 'doji'

    def cs_size(self):
        return abs(self.high - self.low)

    def body_size(self):
        return abs(self.close - self.open)

    def is_bullish(self):
        return self.type() == 'bullish'

    def is_bearish(self):
        return self.type() == 'bearish'

    def is_doji(self):
        return self.type() == 'doji'

    def cs_body_ratio(self):
        return self.body_size() / self.cs_size()

    # define a method that determines numaricly, where the body is placed in the candlestick # anderer Ansatz suchen!!
    def body_position(self):
        if self.open < self.close:
            return (self.open - self.low) / self.cs_size()
        else:
            return (self.close - self.low) / self.cs_size()




############################################################################################################################################################





class CandleStickFrame:
    def __init__(self, open, high, low, close):
        open, high, low, close = self._validate_input(open, high, low, close)
        self.candle_sticks = [CandleStick(o, h, l, c) for o, h, l, c in zip(open, high, low, close)]
        self.df = pd.DataFrame({"open": open, "high": high, "low": low, "close": close})
        self._bullish_count, self._bearish_count, self._doji_count = self._type_count()

    def __repr__(self):
        return f"CandleFrame({self.df})"

    def __str__(self):
        return f"{self.df}"

    def __len__(self):
        return len(self.candle_sticks)

    def __getitem__(self, index):
        return self.candle_sticks[index]

    def __iter__(self):
        return iter(self.candle_sticks)

    def __reversed__(self):
        return reversed(self.candle_sticks)

    @staticmethod
    def _validate_input(open, high, low, close):
        if not isinstance(open, list) and not isinstance(open, np.ndarray) and not isinstance(open, pd.core.series.Series):
            raise TypeError("open must be list, np.ndarry, or pd.core.series.Series not {}".format(type(open)))
        if not isinstance(high, list) and not isinstance(high, np.ndarray) and not isinstance(high, pd.core.series.Series):
            raise TypeError("high must be list, np.ndarry, or pd.core.series.Series not {}".format(type(high)))
        if not isinstance(low, list) and not isinstance(low, np.ndarray) and not isinstance(low, pd.core.series.Series):
            raise TypeError("low must be list, np.ndarry, or pd.core.series.Series not {}".format(type(low)))
        if not isinstance(close, list) and not isinstance(close, np.ndarray) and not isinstance(close, pd.core.series.Series):
            raise TypeError("close must be list, np.ndarry, or pd.core.series.Series not {}".format(type(close)))
        open = list(open)
        high = list(high)
        low = list(low)
        close = list(close)
        if len(open) != len(high) or len(open) != len(low) or len(open) != len(close):
            raise ValueError("open, high, low, and close must be the same length")
        if not all(isinstance(x, (int, float)) for x in open):
            raise TypeError("open must be list of int or float not {}".format(type(open)))
        if not all(isinstance(x, (int, float)) for x in high):
            raise TypeError("high must be list of int or float not {}".format(type(high)))
        if not all(isinstance(x, (int, float)) for x in low):
            raise TypeError("low must be list of int or float not {}".format(type(low)))
        if not all(isinstance(x, (int, float)) for x in close):
            raise TypeError("close must be list of int or float not {}".format(type(close)))
        return open, high, low, close

    def _type_count(self):
        bullish = 0
        bearish = 0
        doji = 0
        for candle in self.candle_sticks:
            if candle.type() == 'bullish':
                bullish += 1
            elif candle.type() == 'bearish':
                bearish += 1
            elif candle.type() == 'doji':
                doji += 1
        return bullish, bearish, doji

    def _bullish_ratio(self):
        return self._bullish_count / len(self.candle_sticks)

    def _bearish_ratio(self):
        return self._bearish_count / len(self.candle_sticks)

    def _doji_ratio(self):
        return self._doji_count / len(self.candle_sticks)

    def type_ratio(self):
        return 'bullish: {:.2f}, bearish: {:.2f}, doji: {:.2f}'.format(self._bullish_ratio(), self._bearish_ratio(), self._doji_ratio())




############################################################################################################################################################





class CandleStickPattern:
    def __init__(self, candle_stick_frame):
        self.candle_stick_frame = candle_stick_frame
        self._average_candle_stick_size = self.average_candle_stick_size()
        self._average_body_size = self.average_body_size()

    def average_candle_stick_size(self):
        return sum([candle.cs_size() for candle in self.candle_stick_frame]) / len(self.candle_stick_frame)

    def average_body_size(self):
        return sum([candle.body_size() for candle in self.candle_stick_frame]) / len(self.candle_stick_frame)

    def trend(self, index, window=10):
        if len(self.candle_stick_frame) < window:
            raise ValueError("window must be less than or equal to the length of the candle stick frame")
        if window < 1:
            raise ValueError("window must be greater than 0")
        if index < 0:
            return None
        trend = 0
        for cs in self.candle_stick_frame[index-window:index]:
            if cs.type() == 'bullish':
                trend += cs.cs_size()
            elif cs.type() == 'bearish':
                trend -= cs.cs_size()
        if trend > 0:
            return 'up'
        elif trend < 0:
            return 'down'
        else:
            return None

    # Bullish Reversal Candlestick Patterns:

    # Bullish Engulfing (3)

    # Bullish Marubozu (6)
    def _is_bullish_marubozu(self, trend, cs, cs_body_ratio=0.9):
        if trend is None:
            return None
        else:
            return cs.is_bullish() and cs.cs_body_ratio() >= cs_body_ratio and trend == 'down'

    def is_bullish_marubozu(self, trend_window=10, cs_body_ratio=0.9):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            cs = self.candle_stick_frame[i]
            results.append(self._is_bullish_marubozu(trend, cs, cs_body_ratio))
        return results

    # Morning Star (4)
    def _is_morning_star(self, trend, cs_m2, cs_m1, cs, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                         cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        if trend is None:
            return None
        elif trend == 'down':
            if cs_m2.is_bearish() and cs_m2.cs_body_ratio() >= cs_m2_body_ratio and cs_m2.cs_size() >= cs_m2_relative_size * self._average_candle_stick_size:
                if cs_m1.cs_size() <= cs_m1_relative_size * self._average_candle_stick_size:
                    if cs.is_bullish() and cs.cs_body_ratio() >= cs_body_ratio and cs.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                        return True
        return False

    def is_morning_star(self,trend_window=10, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                        cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-3, trend_window)
            cs_m2 = self.candle_stick_frame[i-2]
            cs_m1 = self.candle_stick_frame[i-1]
            cs = self.candle_stick_frame[i]
            results.append(self._is_morning_star(trend, cs_m2, cs_m1, cs, cs_m2_body_ratio, cs_body_ratio,
                                                     cs_m2_relative_size, cs_m1_relative_size, cs_relative_size))
        return results

    # Bearish Reversal Candlestick Patterns:

    # Bearish Marubozu (19)
    def _is_bearish_marubozu(self, trend, cs, cs_body_ratio=0.9):
        if trend is None:
            return None
        else:
            return cs.is_bearish() and cs.cs_body_ratio() >= cs_body_ratio and trend == 'up'

    def is_bearish_marubozu(self, trend_window=10, cs_body_ratio=0.9):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            cs = self.candle_stick_frame[i]
            results.append(self._is_bearish_marubozu(trend, cs, cs_body_ratio))
        return results

    # Evening Star (17)
    def _is_evening_star(self, trend, cs_m2, cs_m1, cs, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                         cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        if trend is None:
            return None
        elif trend == 'up':
            if cs_m2.is_bullish() and cs_m2.cs_body_ratio() >= cs_m2_body_ratio and cs_m2.cs_size() >= cs_m2_relative_size * self._average_candle_stick_size:
                if cs_m1.cs_size() <= cs_m1_relative_size * self._average_candle_stick_size:
                    if cs.is_bearish() and cs.cs_body_ratio() >= cs_body_ratio and cs.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                        return True
        return False

    def is_evening_star(self,trend_window=10, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                        cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-3, trend_window)
            cs_m2 = self.candle_stick_frame[i-2]
            cs_m1 = self.candle_stick_frame[i-1]
            cs = self.candle_stick_frame[i]
            results.append(self._is_evening_star(trend, cs_m2, cs_m1, cs, cs_m2_body_ratio, cs_body_ratio,
                                                     cs_m2_relative_size, cs_m1_relative_size, cs_relative_size))
        return results


In [2]:
# load data
df = pd.read_csv('EURUSD_H1.csv')
df

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,Adj Close,<TICKVOL>,<VOL>,<SPREAD>
0,2015.03.09,14:00:00,1.08603,1.08681,1.08460,1.08578,4992,4.480300e+09,10
1,2015.03.09,15:00:00,1.08584,1.08731,1.08545,1.08582,5974,1.009885e+10,11
2,2015.03.09,16:00:00,1.08582,1.08640,1.08452,1.08454,6175,8.667100e+09,9
3,2015.03.09,17:00:00,1.08454,1.08590,1.08406,1.08456,6638,8.587250e+09,0
4,2015.03.09,18:00:00,1.08455,1.08543,1.08376,1.08465,4574,5.572700e+09,7
...,...,...,...,...,...,...,...,...,...
45774,2022.07.19,18:00:00,1.02450,1.02543,1.02305,1.02382,28840,8.652000e+09,8
45775,2022.07.19,19:00:00,1.02382,1.02445,1.02293,1.02398,20792,6.237600e+09,8
45776,2022.07.19,20:00:00,1.02398,1.02435,1.02340,1.02349,17324,5.197200e+09,8
45777,2022.07.19,21:00:00,1.02349,1.02361,1.02251,1.02314,19953,5.985900e+09,8


In [3]:
# create candlestick frame
candle_stick_frame = CandleStickFrame(df['<OPEN>'], df['<HIGH>'], df['<LOW>'], df['Adj Close'])
len(candle_stick_frame)

45779

In [4]:
res = CandleStickPattern(candle_stick_frame).is_morning_star()
count = 0
for r in res:
    if r:
        count += 1
print(count)


79


In [10]:
candle_pattern = CandleStickPattern(candle_stick_frame)
df['is_bullish_marubozu'] = candle_pattern.is_bullish_marubozu()
df['is_bearish_marubozu'] = candle_pattern.is_bearish_marubozu()
df['is_morning_star'] = candle_pattern.is_morning_star()
df['is_evening_star'] = candle_pattern.is_evening_star()
df.describe()

,<OPEN>,<HIGH>,<LOW>,Adj Close,<TICKVOL>,<VOL>,<SPREAD>
count,45779.000000,45779.000000,45779.000000,45779.000000,45779.000000,4.577900e+04,45779.000000
mean,1.134686,1.135455,1.133943,1.134688,19558.567050,6.246788e+09,13.874047
std,0.047305,0.047248,0.047351,0.047307,19805.414961,7.020649e+09,3.274206
min,0.998000,0.999900,0.995150,0.997990,7.000000,0.000000e+00,0.000000
25%,1.101940,1.102710,1.101330,1.101940,4308.000000,2.979672e+09,14.000000
50%,1.128520,1.129300,1.127820,1.128520,14014.000000,5.022463e+09,15.000000
75%,1.173600,1.174230,1.172990,1.173600,29923.000000,7.517674e+09,15.000000
max,1.254840,1.255480,1.254280,1.254840,285485.000000,3.393580e+11,59.000000
